In [17]:
# Importando o módulo de acesso ao SQLite
import sqlite3

# Criando uma conexão e o banco
conexao = sqlite3.connect('monografia.db') 

# Criando um cursor
cursor = conexao.cursor()

In [2]:
# Cria a tabela Tab_averageRating e seus campos
cursor.execute( 'create table if not exists Tab_averageRating '\
'(Titulo_id varchar(10) primary key, '\
'averageRating REAL, '\
'numVotes INTEGER)')

In [3]:
#função de insert na tabela Tab_averageRating
sql = 'INSERT INTO Tab_averageRating (Titulo_id, averageRating, numVotes) VALUES (?, ?, ?)'

In [4]:
#Importar o modulo csv para pegar o arquivo .tsv separado por tabulação
import csv

#abri o arquivo e delimitar por Tabulação
with open("C:/Users/rodri/Documents/Fundamentos_Python/Dados_Monografia/data.tsv",'r') as tsv:
    data_tsv = csv.reader(tsv, delimiter ='\t')
    
    #Pula o cabeçalho do arquivo
    next(data_tsv)
    
    #chama o arquivo e a função de inserção(sql)
    for line in data_tsv:
        cursor.execute(sql,line)

# comita a inserção
conexao.commit()

In [5]:
# Cria a tabela Tab_genero_dos_filmes e seus campos
cursor.execute( 'create table if not exists Tab_genero_dos_filmes '\
'(Titulo_id varchar(10), titleType varchar(50), isAdult INTEGER, duracao INTEGER, startYear INTEGER, generes varchar(100))')

In [6]:
#função de insert na tabela Tab_averageRating
sql2 = 'INSERT INTO Tab_genero_dos_filmes (Titulo_id, titleType, isAdult, duracao, startYear, generes) VALUES (?, ?, ?, ?, ?, ?)'

In [7]:
import csv
#abri o arquivo e delimitar por Tabulação
with open("C:/Users/rodri/Documents/Fundamentos_Python/Dados_Monografia/data2.tsv",encoding='utf8') as tsv:
    data2_tsv = csv.DictReader(tsv, delimiter ='\t')   
    
    for row in data2_tsv:
        linhas = [row['tconst'], row['titleType'],row['isAdult'],row['runtimeMinutes'],row['startYear'],row['genres']]
        cursor.execute(sql2,linhas)
        
# comita a inserção
conexao.commit()

In [8]:
# Cria a tabela Tab_temporadas_episodios e seus campos
cursor.execute( 'create table if not exists Tab_temporadas_episodios '\
'(Titulo_id varchar(10) primary key, seasonNumber INTEGER, episodeNumber INTEGER)')

In [9]:
#função de insert na tabela Tab_temporadas_episodios
sql3 = 'INSERT INTO Tab_temporadas_episodios (Titulo_id, seasonNumber, episodeNumber) VALUES (?, ?, ?)'

In [10]:
import csv
#abri o arquivo e delimitar por Tabulação
with open("C:/Users/rodri/Documents/Fundamentos_Python/Dados_Monografia/data3.tsv",encoding='utf8') as tsv:
    data3_tsv = csv.DictReader(tsv, delimiter ='\t')   
    
    for row in data3_tsv:
        linhas = [row['tconst'], row['seasonNumber'],row['episodeNumber']]
        cursor.execute(sql3,linhas)
        #print(linhas)

# comita a inserção
conexao.commit()

In [5]:
# Join das duas tabelas
cursor.execute('select Tab_averageRating.Titulo_id, averageRating, numVotes, '\
               'titleType, isAdult, startYear, CASE duracao WHEN "\\N" THEN 0 ELSE duracao END as duracao,  generes, '\
               'CASE seasonNumber WHEN "\\N" THEN 0 ELSE seasonNumber END as seasonNumber,  '\
               'CASE episodeNumber WHEN "\\N" THEN 0 ELSE episodeNumber END as episodeNumber '\
               'from Tab_averageRating INNER JOIN Tab_genero_dos_filmes '\
                                   'ON Tab_averageRating.Titulo_id = Tab_genero_dos_filmes.Titulo_id '\
                                      'INNER JOIN Tab_temporadas_episodios '\
                                   'ON Tab_averageRating.Titulo_id = Tab_temporadas_episodios.Titulo_id')

# Recupera os resultados
Join_Tab_aveRating_genFilmes_tempEpisodis = cursor.fetchall()

In [11]:
# Cria a tabela Tab_Final e seus campos
cursor.execute( 'create table if not exists Tab_Final_Filmes '\
'(Titulo_id varchar(10), averageRating REAL, numVotes INTEGER,'\
               'titleType varchar(50), num_titleType INTEGER, isAdult INTEGER, duracao INTEGER, '\
               ' startYear INTEGER, generes varchar(100),'\
               ' seasonNumber INTEGER, episodeNumber INTEGER)')

In [12]:
#função de insert na tabela final
cursor.execute( 'INSERT INTO Tab_Final_Filmes select Tab_averageRating.Titulo_id, averageRating, numVotes, '\
    'titleType, CASE titleType When "short" Then 1 When "movie" Then 2 When "tvMovie" Then 3 '\
                        'When "tvSeries" Then 4 When "tvEpisode" Then 5 When "tvShort" Then 6 '\
                        'When "tvMiniSeries" Then 7 When "tvSpecial" Then 8 When "video" Then 9 Else 10 End AS num_titleType, '\
               'isAdult,  CASE duracao WHEN "\\N" THEN 0 ELSE duracao END as duracao, startYear, '\
    'CASE generes WHEN "\\N" THEN 0 ELSE generes END as generes, '\
    'CASE seasonNumber WHEN "\\N" THEN 0 WHEN "" THEN "0" ELSE seasonNumber END as seasonNumber,  '\
    'CASE episodeNumber WHEN "\\N" THEN 0 WHEN "" THEN "0" ELSE episodeNumber END as episodeNumber '\
            'from Tab_averageRating left JOIN Tab_genero_dos_filmes '\
                                   'ON Tab_averageRating.Titulo_id = Tab_genero_dos_filmes.Titulo_id '\
                            'left JOIN Tab_temporadas_episodios '\
                                   'ON Tab_averageRating.Titulo_id = Tab_temporadas_episodios.Titulo_id')

# comita a inserção
conexao.commit()

In [13]:
#função que altera os valores nulos das colunas seasonNumber e episodeNumber.
cursor.execute("UPDATE Tab_Final_Filmes SET seasonNumber = 0 WHERE seasonNumber IS NULL")
cursor.execute("UPDATE Tab_Final_Filmes SET episodeNumber = 0 WHERE episodeNumber IS NULL")

# comita a inserção
conexao.commit()


In [14]:
#função que altera os valores os valores do campo generes
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Acao' WHERE substr(generes,1,5) = 'Actio'")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Adulto' WHERE substr(generes,1,5) = 'Adult'")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Animacao' WHERE substr(generes,1,5) = 'Anima'")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Aventura' WHERE substr(generes,1,5) in ('Adven','Game-') ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Biografia' WHERE substr(generes,1,5) = 'Biogr' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Comedia' WHERE substr(generes,1,5) in ('Comed','Talk-') ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Crime' WHERE substr(generes,1,5) = 'Crime' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Curtas' WHERE substr(generes,1,5) = 'Short' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Documentario' WHERE substr(generes,1,5) in ('Docum','Histo') ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Drama' WHERE substr(generes,1,5) = 'Drama' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Esportes' WHERE substr(generes,1,5) = 'Sport' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Familia' WHERE substr(generes,1,5) = 'Famil' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Faroeste' WHERE substr(generes,1,5) = 'Weste' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Ficcao Cientifica' WHERE substr(generes,1,5) = 'Sci-F' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Guerra' WHERE substr(generes,1,5) in ('War', 'War,W')")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Fantasia' WHERE substr(generes,1,5) in ('Myste', 'Fanta')")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Musical' WHERE substr(generes,1,5) = 'Music' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Noticias' WHERE substr(generes,1,5) in ('News', 'News,')")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Reality' WHERE substr(generes,1,5) = 'Reali' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Romance' WHERE substr(generes,1,5) = 'Roman'")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Suspense' WHERE substr(generes,1,5) = 'Thril' ")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Terror' WHERE substr(generes,1,5) = 'Horro'")
cursor.execute("UPDATE Tab_Final_Filmes SET  generes = 'Não Identificado' WHERE substr(generes,1,5) in ('Film-', '0')")



# comita a inserção
conexao.commit()

In [15]:
#função que altera os valores os valores do campo e adulto, pois tem várias classificações que são adultas

cursor.execute("UPDATE Tab_Final_Filmes SET  isAdult = 1 WHERE generes in ('Biografia', 'Crime', 'Documentário', 'Drama', 'Guerra', 'Notícias', 'Reality', 'Talk-Show', 'Terror') ")


In [18]:
#exportando um arquvio de amostra no formato csv (;) para uma AASs para short-tvShort-video(1,6,9)
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra_AASs_short-tvShort-video.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, startYear, generes '\
               ' from Tab_Final_Filmes where titleType <> "videoGame" and  '\
               ' generes <> "Não Identificado" and num_titleType in (1,6,9) and  numVotes > 5 '\
               ' and duracao > 0 ORDER BY RANDOM() limit 470')
    table = cursor.fetchall()
    csv_dados.writerow(["Media_Classsificacao","Numero_de_Votos","Classificacao_Tipo_de_Titulo","E_Adulto","Minutos_de_Duracao",
                        "Ano_da_Producaao","Genero"])
    csv_dados.writerows(table)

In [21]:
#exportando um arquvio de amostra no formato csv (;) para uma AASs para movie(2)
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra_AASs_movie.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, startYear, generes '\
               ' from Tab_Final_Filmes where titleType <> "videoGame" and  '\
               ' generes <> "Não Identificado" and num_titleType in (2) and  numVotes > 5 '\
               ' and duracao > 0   ORDER BY RANDOM() limit 646')
    table = cursor.fetchall()
    csv_dados.writerow(["Media_Classsificacao","Numero_de_Votos","Classificacao_Tipo_de_Titulo","E_Adulto","Minutos_de_Duracao",
                        "Ano_da_Producaao","Genero"])
    csv_dados.writerows(table)

In [20]:
#exportando um arquvio de amostra no formato csv (;) para uma AASs para tvMovie-tvSpecial(3,7,8)
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra_AASs_tvMovie-tvMiniserie-tvSpecial.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, startYear, generes '\
               ' from Tab_Final_Filmes where titleType <> "videoGame" and  '\
               ' generes <> "Não Identificado" and num_titleType in (3,7,8) and  numVotes > 5 '\
               ' and duracao > 0   ORDER BY RANDOM() limit 158')
    table = cursor.fetchall()
    csv_dados.writerow(["averageRating","numVotes","num_titleType","isAdult","duracao","startYear",
                        "generes"])
    csv_dados.writerows(table)

In [19]:
#exportando um arquvio de amostra no formato csv (;) para uma AASs para tvEpisode-tvSeries (5,4)
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra_AASs_tvEpisode-tvSeries.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, startYear, generes, seasonNumber '\
               ',episodeNumber from Tab_Final_Filmes where titleType <> "videoGame" and  '\
               ' generes <> "Não Identificado" and num_titleType in (5,4) and  numVotes > 5 '\
               ' and duracao > 0   ORDER BY RANDOM() limit 1353')
    table = cursor.fetchall()
    csv_dados.writerow(["Media_Classsificacao","Numero_de_Votos","Classificacao_Tipo_de_Titulo","E_Adulto","Minutos_de_Duracao", "Ano_da_Producao",
                        "Genero","Numero_de_Temporadas","Numero_de_Episodios"])
    csv_dados.writerows(table)

In [ ]:
#Teste para ver os valores que virão
import pandas as pd
file_name = open("amostra_AASs.csv")
df = pd.read_csv(file_name,header=None,delimiter = ';')
df.head()

In [ ]:
# Pega amostras por Amostragens Estratificada Proporcional gerando um objeto que carrega a concatenação das amostras.
# Amostra AE porporcional com AAS sem reposição para categoria movie
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "movie" and  generes <> 0  ORDER BY RANDOM() limit 646')
arq_movie = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria short
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "short" and  generes <> 0 ORDER BY RANDOM() limit 321')
arq_short = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvEpisode
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvEpisode"  and  generes <> 0 ORDER BY RANDOM() limit 1180')
arq_tvEpisode = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvMiniSeries
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvMiniSeries"  and  generes <> 0 ORDER BY RANDOM() limit 24')
arq_tvMiniSeries = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvMovie
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvMovie"  and  generes <> 0 ORDER BY RANDOM() limit 114')
arq_tvMovie = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvSeries
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvSeries"  and  generes <> 0 ORDER BY RANDOM() limit 173')
arq_tvSeries = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvShort
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvShort"  and  generes <> 0 ORDER BY RANDOM() limit 8')
arq_tvShort = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria tvSpecial
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "tvSpecial"  and  generes <> 0 ORDER BY RANDOM() limit 20')
arq_tvSpecial = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para categoria vídeo
cursor.execute('select averageRating, numVotes, num_titleType, isAdult, duracao, generes, seasonNumber,episodeNumber '\
               ' from Tab_Final_Filmes where titleType = "video"  and  generes <> 0 ORDER BY RANDOM() limit 141')
arq_video = cursor.fetchall()

# Amostra AE porporcional com AAS sem reposição para o Arquivo final
arq_amostra_AE = arq_movie + arq_short + arq_tvEpisode + arq_tvMiniSeries + \
                arq_tvMovie + arq_tvSeries + arq_tvShort + arq_tvSpecial + arq_video

In [ ]:
#exportando um arquvio de amostra no formato csv(;) para uma AE com AASs
import csv
import random
with open('amostra_AE.csv', 'w', newline = '') as file:
    csv_dados_2 = csv.writer(file, delimiter = ';')
    csv_dados_2.writerow(["averageRating","numVotes","num_titleType","isAdult","E-duracao",
                          "generes","seasonNumber","episodeNumber"])
    csv_dados_2.writerows(arq_amostra_AE)

In [ ]:
#Teste para ver os valores que virão
import pandas as pd
file_name = open("amostra_AE.csv")
df = pd.read_csv(file_name,header=None,delimiter = ';')
df.head()

In [16]:
# comandos fecha conexão e fecha cursor.
cursor.close()
conexao.close()

In [ ]:
#Função deleta os dados do banco
cursor.execute("DELETE FROM Tab_Final_Filmes")
# comita a inserção
conexao.commit()

In [3]:
#Função deleta os dados do banco
cursor.execute("DROP TABLE Tab_Final_Filmes")
# comita a inserção
conexao.commit()

In [20]:
cursor.execute('select distinct count(*) '\
               ' from Tab_Final_Filmes where generes <> "Não Identificado" and num_titleType not in (10)')
              

dados = cursor.fetchall()
(dados)

[(910991,)]

In [15]:
#exportando um arquvio de amostra no formato csv (;) para uma AASs - Este arquivos serão usados para calculo do 
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra_tvMiniserie.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('select averageRating from Tab_Final_Filmes where titleType <> "videoGame" and generes <> 0 '\
                   ' and num_titleType in (7)')
    table = cursor.fetchall()
    csv_dados.writerow(["averageRating"])
    csv_dados.writerows(table)

In [ ]:
#achar os desvios padrões de cada variavel tipo de filme
#exportando um arquvio de amostra no formato csv(;)
import csv
import random

#Abri ou cria um arvivo e salva as amostras da tabela no arquivo.csv criado 'amostra.csv'
with open('amostra9.csv', 'w', newline = '') as file:
    csv_dados = csv.writer(file, delimiter = ';')
    #Retorna o tamanho limite de amostra randomicamente
    cursor.execute('Select averageRating From Tab_Final_Filmes where num_titleType =9 '\
                   ' and generes <> 0 and num_titleType <> 10')
    table = cursor.fetchall()
    csv_dados.writerows(table)